# Teemo Ultimate Progress
In Riot Game's League of Legends, the champion "Teemo" is one of my most played. Teemo is infamous for the mushroom traps (or shrooms) in his kit and they're commonly targeted by enemy players as objectives to destroy in game. In this project, I aim to determine if the number of shrooms Teemo places and the number of shrooms that are destroyed affect Teemo's kill participation at the end of a game of league. 

I have sourced data used in this project through the Riot API; specifically, I have obtained my recent match history in which I have played as Teemo.

# Code

In [ ]:
library(tidyverse)

Loading in the processed data

In [ ]:
data <- read_csv("../data/summary.data") %>%
    map_df(rev) %>%
    rowid_to_column("index")
head(data)

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)

Creating EDA visualizations for the various Shroom time series

In [ ]:
shroom_count <- ggplot(data) +
    geom_point(aes(x=index, y=shrooms_created, color="Shrooms Created"), color = "dark green", alpha = 0.4) +
    geom_line(aes(x=index, y=shrooms_created, color="Shrooms Created"), color = "dark green", alpha = 0.8) +
    geom_point(aes(x=index, y=shrooms_destroyed, color="Shrooms Destroyed"),color = "purple" , alpha = 0.4) +
    geom_line(aes(x=index, y=shrooms_destroyed, color="Shrooms Destroyed"),color = "purple", alpha = 0.8) +
    labs(x = "Match No.", y = "Shroom Count", color = "Shroom Count Source") +
    ggtitle("Shroom Count Time Series") + 
    theme(text = element_text(size = 20))
shroom_count

In [ ]:
shroom_damage <- ggplot(data) +
    geom_point(aes(x=index, y=shroom_damage), color = "dark green", alpha = 0.4) +
    geom_line(aes(x=index, y=shroom_damage), color = "dark green", alpha = 0.8) +
    labs(x = "Match No.", y = "Shroom Damage") +
    ggtitle("Shroom Damage Time Series") + 
    theme(text = element_text(size = 20))
shroom_damage

From the basic visualizations against time, there does not seem to be a clear seasonal effect or trend between matches. Therefore, it may not be appropriate to use time series regression analysis to determine future progress.

In [ ]:
shrooms_created_vs_damage <- ggplot(data) +
    geom_point(aes(x=shrooms_created, y=shroom_damage), color = "dark green", alpha = 0.8, size = 3) +
    labs(x = "Shrooms Created", y = "Shroom Damage") +
    ggtitle("Shrooms Created vs Shroom Damage")+
    theme(text = element_text(size = 20))
shrooms_created_vs_damage

In [ ]:
shrooms_destroyed_vs_damage <- ggplot(data) +
    geom_point(aes(x=shrooms_destroyed, y=shroom_damage), color = "dark green", alpha = 0.8, size = 3) +
    labs(x = "Shrooms Destroyed", y = "Shroom Damage") +
    ggtitle("Shrooms Destroyed vs Shroom Damage")+
    theme(text = element_text(size = 20))
shrooms_destroyed_vs_damage

From a quick observation, it seems there is a weak correlation between damage and shrooms created/destroyed. Despite being a weak correlation, it may be useful to analyze this data. 

With this this information in mind, I declare the following alternative hypothesis as the basis of this research project: The effect of Teemo Shrooms in kill participations is dependent on the effect of Teemo Shrooms created and destroyed. We will use a multiple linear regression analysis to determine the reality of this hypothesis at a 5% significance level.

In other words:

$H_0: \beta_{1}=\beta_{2}=0$ where $\beta_{1}, \beta_{2}$ are the regression coefficients for Shrooms Created and Shrooms Destroyed respectively

$H_A: \beta_{1}=\beta_{2}\neq 0$

We can use the general linear regression model $Y = \beta_{0} + \beta_{1}X + \beta_{2}Y$ where $\beta_{0}$ is the intercept coefficient

In [ ]:
linear_model <- lm(kill_participation ~ shrooms_created + shrooms_destroyed, data=data)
linear_model

However, it may still be inappropriate to use the general linear regression model. As such, we shall create a residual plot.

In [ ]:
plot(resid(linear_model))
abline(0,0)

Since we don't have much data, it's hard to make conclusions solely based off of the residual plot; we can't determine if there exists a pattern in the residuals so we can't rule out a linear model. To obtain more information about our model, we can generate an in-depth summary.

In [ ]:
summary(linear_model)

Both R-Squared values seem to indicate that our model is inappropriate for our data. The multiple R-squared value suggests that 39.9% of the variation in kill participation is explained by the linear model whereas the adjusted R-squared value suggests that 33.2% of the variation in kill participation is explained by the linear model. In either case, it should be transformed in an attempt to find a better fitting model.

From the residual plot, it can be argued that a faint "funnel" shape can be seen. This suggests that a square-root or logarithmic transformation may be required on kill participation

In [ ]:
log_model <- lm(log(kill_participation) ~ shrooms_created + shrooms_destroyed, data=data)
plot(resid(log_model))
abline(0,0)

In [ ]:
sqrt_model <- lm(sqrt(kill_participation) ~ shrooms_created + shrooms_destroyed, data=data)
plot(resid(sqrt_model))
abline(0,0)

From basic observation the residual plots of the transformed models, it seems that both transformations resulted in a "more random" plot. To decide the better model, we must once again look at the R^2 values

In [ ]:
summary(log_model)
summary(sqrt_model)

For both the multiple R-squared and Adjusted R-squared values, the square-root transformed model seems to be the best choice between the two models. As such, I choose to adopt the logarithmic model.

# Results and Discussion

From the variable p-values, we can see that the number of shrooms destroyed are not statistically significant ($0.13932 > 0.05$) which means that the number of shrooms destroyed does not significantly affect kill participation. 

However, the p-value for the number of shrooms created is statistically significant ($0.00317 < 0.05$) which means that the number of shrooms created does affect kill participation. 

As such we reject the null hypothesis that shrooms created and destroyed do not affect kill participation in favour of the alternative hypothesis that shrooms and destroyed affect kill participation.

In conclusion, the model suggests there is a positive correlation between shrooms created and kill participation a given game where I play as the League of Legends champion "Teemo". As a crucial part of Teemo's ability kit and identity, it is unsurprising that utilizing more shrooms in a game results in a higher kill participation. 

Although, it is important to note that there is only a weak correlation between the predictors and kill participation since the estimator for shrooms created is close to 0. Despite having a statistically significant effect on the regression model, they do not affect kill participation by much.

With this in mind, there are many areas which could be improved on for this project. A large drawback of the Riot API is that we can only obtain a player's previous 100 matches. Therefore, it is hard to confidently apply any statistical testing due to the small amounts of data. One possible work around to this is for me to play multiple consecutive games of Teemo to maximize the amount of data available. Alternatively, an automatic scraper could be used once every few months to record a match history greater than the limit provided in the Riot API. This could allow for even more data without the need for the previous solution.